In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',  categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',  categories=categories)

# pprint(newsgroups_train.data[0])

num_train = len(newsgroups_train.data)
num_test  = len(newsgroups_test.data)

vectorizer = TfidfVectorizer(max_features=20)

X = vectorizer.fit_transform( newsgroups_train.data + newsgroups_test.data )
X_train = X[0:num_train, :]
X_test = X[num_train:num_train+num_test,:]

Y_train = newsgroups_train.target
Y_test = newsgroups_test.target

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

((2034, 20), (2034,))
((1353, 20), (1353,))


In [2]:
# print(X)

print(newsgroups_test.filenames)

['/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/sci.space/62410'
 '/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/comp.graphics/38808'
 '/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/comp.graphics/40062'
 ...
 '/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/talk.religion.misc/84302'
 '/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/comp.graphics/38839'
 '/Users/yzh/scikit_learn_data/20news_home/20news-bydate-test/comp.graphics/38973']


In [3]:
# %% 4
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [4]:
# %% 7
# Helper function to evaluate the total loss on the dataset
def calculate_loss(model, X, y):
    W1, b1, W2, b2, W3, b3, W4, b4 = model['W1'], model['b1'], model['W2'], model['b2'],model['W3'], model['b3']model['W4'], model['b4']
    z1 = X.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    a2 = np.tanh(z2)
    z3 = a2.dot(W3) + b3
 
    a3 = np.tanh(z3)
    z4 = a3.dot(W4) + b4
    
    exp_scores = np.exp(z3)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # Calculating the loss
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    # Add regulatization term to loss (optional)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss


# %% 8
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2, W3, b3, W4,b4 = model['W1'], model['b1'], model['W2'], model['b2'],model['W3'], model['b3']model['W4'], model['b4']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    a2 = np.tanh(z2)
    z3 = a2.dot(W3) + b3
    
    exp_scores = np.exp(z3)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)


# %% 16
# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(X, y, nn_hdim, epsilon, reg_lambda, num_passes=20000,  print_loss=False):
 
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    
    W2 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_hdim))
    
    W3 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b3 = np.zeros((1, nn_output_dim))
    
    # This is what we return at the end
    model = {}
    
    # Gradient descent. For each batch...
    for i in range(0, num_passes):

        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        a2 = np.tanh(z2)
        z3 = a2.dot(W3) + b3
        
        exp_scores = np.exp(z3)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW3 = (a2.T).dot(delta3)
        db3 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W3.T) *(1 - np.power(a2, 2))
        
        
        dW2 = (a1.T).dot(delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        delta1 = delta2.dot(W2.T) * (1 - np.power(a1, 2))
#         print("X")
#         print(X.shape)
#         print("delta")
#         print(delta2.shape)

        dW1 =  X.T.dot(delta1) #np.dot(X, delta2)
        db1 = np.sum(delta1, axis=0)
        
#         print("dW1")
#         print(dW1.shape)
        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        
#         print("W1")
#         print(W1.shape)
        dW1 += reg_lambda * W1

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
          print("Loss after iteration %i: %f" %(i, calculate_loss(model,X, y)))
    
    return model




In [5]:

nn_input_dim = 20 # input layer dimensionality
nn_output_dim = 20 # output layer dimensionality


In [8]:
# %% 17
# Build a model with a 3-dimensional hidden layer

num_examples, input_dim = X_train.shape
epsilon = 0.001
reg_lambda = 0.00
epochs = 10000

for i, nn_hdim in enumerate([3,4,6,8,15,16,17,18,19,input_dim]): #
    model = build_model(X_train, Y_train, nn_hdim, epsilon, reg_lambda, epochs, print_loss=True)

    n_correct = 0
    n_test = X_test.shape[0]
    for n in range(n_test):
        x = X_test[n,:]
        yp = predict(model, x)
        if yp == Y_test[n]:
            n_correct += 1.0
    print('nn_hdim：%d  Accuracy %f = %d / %d'%(nn_hdim, n_correct/n_test, int(n_correct), n_test))
    print("")


Loss after iteration 0: 2.628085
Loss after iteration 1000: 2.540626
Loss after iteration 2000: 2.537113
Loss after iteration 3000: 2.536102
Loss after iteration 4000: 2.535649
Loss after iteration 5000: 2.535163
Loss after iteration 6000: 2.534543
Loss after iteration 7000: 2.533648
Loss after iteration 8000: 2.532713
Loss after iteration 9000: 2.532091
nn_hdim：3  Accuracy 0.175905 = 238 / 1353

Loss after iteration 0: 2.836770
Loss after iteration 1000: 2.609308
Loss after iteration 2000: 2.595926
Loss after iteration 3000: 2.591140
Loss after iteration 4000: 2.588449
Loss after iteration 5000: 2.586048
Loss after iteration 6000: 2.584475
Loss after iteration 7000: 2.583415
Loss after iteration 8000: 2.582604
Loss after iteration 9000: 2.581933
nn_hdim：4  Accuracy 0.187731 = 254 / 1353

Loss after iteration 0: 2.351954
Loss after iteration 1000: 2.003468
Loss after iteration 2000: 1.990362
Loss after iteration 3000: 1.975742
Loss after iteration 4000: 1.961626
Loss after iteration 50